In [1]:
import os
import json
import itertools
from copy import deepcopy
from random import sample
import pandas as pd

In [2]:
print(os.getcwd())

/Users/lucas/PycharmProjects/Res-IRF4/project/input/policies/interaction_current_pp


In [3]:
variant = {
    'subsidies': 'variant_subsidies.json',
    'restriction_gas': 'variant_ban_boilers.json',
    'carbon_tax': 'variant_carbon_tax.json',
    'obligation': 'variant_mandatory_renovation.json',
    'cee': 'variant_wco.json',
    'zero_interest_loan': 'variant_zil.json'
}

In [4]:
temp_policies, temp_name = {}, {}
for key, f in variant.items():
    with open(f) as file:
        p = json.load(file)
    
    # add option to not implement the policy
    p.update({'no_{}'.format(key): {}})
    
    temp_policies.update({key: list(p.values())})
    temp_name.update({key: list(p.keys())})
    

In [5]:
keys, values = zip(*temp_name.items())
permutations_names = [dict(zip(keys, v)) for v in itertools.product(*values)]
permutations_names = {'S{}'.format(n): v for n, v in enumerate(permutations_names)}

keys, values = zip(*temp_policies.items())
permutations_policies = [dict(zip(keys, v)) for v in itertools.product(*values)]
permutations_policies = {'S{}'.format(n): v for n, v in enumerate(permutations_policies)}

In [6]:
f = '../policies_calibration.json'
with open(f) as file:
    policies_ref = json.load(file)

In [7]:
result = {}
for key, policies in permutations_policies.items():
    # for each scenario, add the reference policies
    temp = deepcopy(policies_ref['policies'])
    
    # add the policies from group
    group = []
    for k, i in policies.items():
        # check if it's a policy or a group of policies
        if 'start' not in i.keys():
            temp.update(i)
            group.append(k)
    # remove the group of policies
    for k in group:
        policies.pop(k)
        
    # add the other policies
    temp.update(policies)
    result.update({key: deepcopy(temp)})

In [8]:
print(len(result.keys()))

384


In [9]:
# sampling results
sampled = False
if sampled:
    number_scenarios = 10
    result_sample = sample(list(result.items()), number_scenarios)
    result_sample = {k: i for k, i in result_sample}

    permutations_names = {k: permutations_names[k] for k in result_sample.keys()}
    pd.DataFrame(permutations_names).T.to_csv('policies_scenarios_description.csv')
else:
    result_sample = result.copy()
    pd.DataFrame(permutations_names).T.to_csv('policies_scenarios_description.csv')

In [10]:
print(result_sample[list(result_sample.keys())[2]])
print(result_sample.keys())

{'carbon_tax': {'start': 2017, 'end': 2051, 'tax': 'project/input/policies/current/carbon_tax_growth.csv', 'emission': 'project/input/policies/current/carbon_tax_emission.csv', 'recycling': None, 'policy': 'carbon_tax'}, 'cite': {'start': 2018, 'end': 2019, 'heater': 'project/input/policies/current/cite_heater_sf.csv', 'insulation': 'project/input/policies/current/cite_insulation_sf.csv', 'cap': 4800, 'policy': 'subsidy_ad_valorem'}, 'cee': {'start': 2017, 'end': 2051, 'cumac_heater': 'project/input/policies/current/cumac_heater_per_year.csv', 'cumac_insulation': 'project/input/policies/current/cumac_insulation_per_year.csv', 'bonus_insulation': {'value': 'project/input/policies/current/cee_bonus_insulation.csv', 'start': 2019, 'end': 2022}, 'bonus_heater': {'value': 'project/input/policies/current/cee_bonus_heater.csv', 'start': 2019, 'end': 2026}, 'coefficient_obligation': 'project/input/policies/current/cee_coefficient_obligation.csv', 'policy': 'subsidy_targeted', 'value': 'project

In [11]:
with open('policies_scenarios.json', 'w') as fp:
    json.dump(result_sample, fp)

In [12]:
# test

In [13]:
with open('policies_scenarios.json') as file:
    policies_test = json.load(file)

In [14]:
policies_test[list(policies_test.keys())[0]]

{'carbon_tax': {'start': 2017,
  'end': 2051,
  'tax': 'project/input/policies/current/carbon_tax_growth.csv',
  'emission': 'project/input/policies/current/carbon_tax_emission.csv',
  'recycling': None,
  'policy': 'carbon_tax'},
 'cite': {'start': 2018,
  'end': 2019,
  'heater': 'project/input/policies/current/cite_heater_sf.csv',
  'insulation': 'project/input/policies/current/cite_insulation_sf.csv',
  'cap': 4800,
  'policy': 'subsidy_ad_valorem'},
 'cee': {'start': 2017,
  'end': 2051,
  'cumac_heater': 'project/input/policies/current/cumac_heater_per_year.csv',
  'cumac_insulation': 'project/input/policies/current/cumac_insulation_per_year.csv',
  'bonus_insulation': {'value': 'project/input/policies/current/cee_bonus_insulation.csv',
   'start': 2019,
   'end': 2022},
  'bonus_heater': {'value': 'project/input/policies/current/cee_bonus_heater.csv',
   'start': 2019,
   'end': 2051},
  'coefficient_obligation': 'project/input/policies/current/cee_coefficient_obligation.csv',
 